In [16]:
#Webscraper
#import libraries
import requests
from bs4 import BeautifulSoup

#get faq q a pairs from swinburne online faq
def get_faqs():
    #sends http request to swinburne online faq webpage to create Beautiful Soup object
    response = requests.get('https://www.swinburneonline.edu.au/faqs/')
    soup = BeautifulSoup(response.content, 'html.parser')
    #grab all elements of class faqs-group and card and assign them to faq cards.
    faqs_cards = soup.select('.faqs-group .card')
    result = []
    #gets questions and answers from faq cards and places them into result list
    for faq in faqs_cards:
        question = faq.select_one('.card-header h5 > div:nth-child(2)')
        answer = faq.select_one('.card-body .content')
        # add to result if question and answer exist
        if question and answer:
            question = question.get_text(strip=True)
            answer = answer.get_text(strip=True)
            result.append((question, answer))
    return result

In [15]:
#combo rephraser

#bart rephraser
import csv
from transformers import BartForConditionalGeneration, BartTokenizer, AutoTokenizer, AutoModelForSeq2SeqLM


#get the faq data
faq_data = get_faqs()

#https://huggingface.co/eugenesiow/bart-paraphrase
bartmodel = BartForConditionalGeneration.from_pretrained("eugenesiow/bart-paraphrase")
barttokenizer = BartTokenizer.from_pretrained("eugenesiow/bart-paraphrase")

#https://huggingface.co/humarin/chatgpt_paraphraser_on_T5_base?text=What+kind+of+support+do+you+offer+to+your+online+students%3F
device = "cpu"
t5tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
t5model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

#https://huggingface.co/ramsrigouthamg/t5-large-paraphraser-diverse-high-quality
rammodel = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
ramtokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")

def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=1, num_beams=10):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    max_length=300,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )

  print(outputs.size())
  # decode the generated sentences using the tokenizer to get them back to text
  return [s.strip() for s in tokenizer.batch_decode(outputs, skip_special_tokens=True)]


#define the csv file path and name
csv_file_path = "faq_data.csv"


#open the csv file for writing
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    #create a csv writer object
    csv_writer = csv.writer(csv_file)

    #write the header row
    csv_writer.writerow(["Question", "Answer", "Label"])

    

    #write each faq as a row in the csv file and add label
    for i, faq in enumerate(faq_data):
        question, answer = faq
        #label of 0 corresponds to in scope
        label = "0"
        csv_writer.writerow([question, answer, label])

    #generates and writes paraphrased strings to csv
    for i, faq in enumerate(faq_data):
        question, answer = faq

        #bart repponses
        bart_paraphrased_question = get_paraphrased_sentences(bartmodel, barttokenizer, question, num_beams=20, num_return_sequences=1)[0]
        bart_paraphrased_answer = get_paraphrased_sentences(bartmodel, barttokenizer, answer, num_beams=20, num_return_sequences=1)[0]

        # write the initial paraphrases to the CSV file
        csv_writer.writerow([bart_paraphrased_question, bart_paraphrased_answer, '0'])

        #t5 paraphrases
        t5_paraphrased_question = get_paraphrased_sentences(t5model, t5tokenizer, question, num_beams=20, num_return_sequences=1)[0]
        t5_paraphrased_answer = get_paraphrased_sentences(t5model, t5tokenizer, answer, num_beams=20, num_return_sequences=1)[0]

        # write the paraphrases to the CSV file
        csv_writer.writerow([t5_paraphrased_question, t5_paraphrased_answer, '0'])

        #ram paraphrases
        ram_paraphrased_question = get_paraphrased_sentences(rammodel, ramtokenizer, question, num_beams=20, num_return_sequences=1)[0]
        ram_paraphrased_answer = get_paraphrased_sentences(rammodel, ramtokenizer, answer, num_beams=20, num_return_sequences=1)[0]

         # write the paraphrases to the CSV file. remove lable string attached by ram bot.
        csv_writer.writerow([ram_paraphrased_question.replace("paraphrasedoutput: ", ""), ram_paraphrased_answer.replace("paraphrasedoutput: ", ""), '0'])

    #copies qa pairs from squad
    validation_squad_path = "validation-squad.csv"
    with open(validation_squad_path, "r", encoding="utf-8") as squad:
        csv_reader = csv.reader(squad)
        next(csv_reader)  # skip the header row
    
        # Loop over each row in the validation-squad CSV file
        for row in csv_reader:
           # Get the question-answer pairs from columns 3 and 6
            question = row[2]
            answer = row[5]
        
            # Write the question-answer pair to the existing CSV file with a label of "1"
            csv_writer.writerow([question, answer, "1"])

    
    csv_file.close()




torch.Size([1, 10])
torch.Size([1, 58])
torch.Size([1, 10])
torch.Size([1, 68])
torch.Size([1, 14])


KeyboardInterrupt: 